In [ ]:
// ROOT::EnableImplicitMT();

In [ ]:
ROOT::RDataFrame dis("dis", "dis.root");

In [ ]:
gStyle->SetOptStat(11111111);

In [ ]:
using ROOT::VecOps::RVec;
using ROOT::VecOps::Map;
using ROOT::VecOps::Filter;

In [ ]:
template<typename T>
auto clones_converter(const TClonesArray &clones) -> RVec<T>{
    RVec<T> items;
    for (auto&& clone : clones) {
        auto item = static_cast<T*>(clone);
        items.emplace_back(*item);
    }
    return items;
};

In [ ]:
auto dis_with_vars = dis
.Define("mu", "*dynamic_cast<TLorentzVector*>(muon[0])")
.Define("products", clones_converter<TLorentzVector>, {"particles"})
.Define("theta", [](const RVec<TLorentzVector>& ps, const TLorentzVector& mu){
    return Map(ps, [&mu](const TLorentzVector& particle){
        return mu.Angle(particle.Vect());
    });
}, {"products", "mu"})
.Define("lifetime", [](const RVec<int>& ps){
    return Map(ps, [](const int& id){
        auto PDG = TDatabasePDG::Instance();
        return PDG->GetParticle(id)->Lifetime();
    });
}, {"pids"})
.Define("gamma", [](const RVec<TLorentzVector>& products){
    return Map(products, [](const TLorentzVector& p){
        return p.Gamma();
    });
}, {"products"})
.Define("beta", [](const RVec<TLorentzVector>& products){
    return Map(products, [](const TLorentzVector& p){
        return p.Beta();
    });
}, {"products"})
.Define("P", [](const RVec<TLorentzVector>& products){
    return Map(products, [](const TLorentzVector& p){
        return TMath::Sqrt(p.Px()*p.Px()+p.Py()*p.Py()+p.Pz()*p.Pz());
    });
}, {"products"})
.Define("E", [](const RVec<TLorentzVector>& products){
    return Map(products, [](const TLorentzVector& p){
        return p.E();
    });
}, {"products"})
.Filter("mu.E()<70", "µ momentum cut 70 GeV")
.Define("ctau", "beta*TMath::C()*gamma*lifetime")
.Define("farEnough", "ctau>2")
.Define("theta_far", "theta[farEnough]")
.Define("hiAngle", "theta>TMath::Pi()/4")
.Define("K_L", "pids==130")
.Define("momentum", "P[hiAngle && farEnough]")
.Define("energy", "E[hiAngle && farEnough]")
.Define("K_L_with_angle", "K_L[hiAngle]")
.Define("K_L_with_angle_and_ctau", "K_L[farEnough && hiAngle]")
.Define("K_L_with_ctau", "K_L[farEnough]")

In [ ]:
auto thetas = dis_with_vars.Histo1D("theta");
auto pids = dis_with_vars.Histo1D("pids");
auto lifetimes = dis_with_vars.Histo1D("lifetime");
auto ctaus = dis_with_vars.Histo1D("ctau");
auto momentum = dis_with_vars.Histo1D("momentum");
auto energy = dis_with_vars.Histo1D("energy");

In [ ]:
auto report = dis_with_vars.Report()

In [ ]:
auto kaons = dis_with_vars.Sum("K_L");
auto dangerous_kaons = dis_with_vars.Sum("K_L_with_angle");
auto very_dangerous_kaons = dis_with_vars.Sum("K_L_with_angle_and_ctau");
auto old_kaons = dis_with_vars.Sum("K_L_with_ctau");

In [ ]:
std::cout << kaons.GetValue() << std::endl;
std::cout << dangerous_kaons.GetValue() << std::endl;
std::cout << very_dangerous_kaons.GetValue() << std::endl;
std::cout << old_kaons.GetValue() << std::endl;

# Explode operation would be great to generate cutflow on tracks/particles

In [ ]:
report->Print();

In [ ]:
11757./4310000.

In [ ]:
gStyle->SetOptStat(0);

In [ ]:
TCanvas c1;
c1.SetLogy();
thetas->Draw();
c1.Draw();

In [ ]:
TCanvas c2;
pids->Draw();
c2.Draw();

In [ ]:
TCanvas c3;
lifetimes->Draw();
c3.Draw();

In [ ]:
TCanvas c4;
c4.SetLogy();
ctaus->Draw();
c4.Draw();

In [ ]:
TCanvas c7;
momentum->Draw();
c7.Draw();

In [ ]:
TCanvas c8("c", "c", 400,400);

In [ ]:
energy->GetXaxis()->CenterTitle(true);
energy->SetTitle("");
energy->GetXaxis()->SetTitle("$E\\;[\\mathrm{GeV}/c^2]$");
energy->GetXaxis()->SetTitleSize(0.04);
energy->Draw();

c8.Draw();

In [ ]:
c8.SaveAs("dangerous_energies.png");
c8.SaveAs("dangerous_energies.tex");

In [ ]:
c7.SaveAs("dangerous_momenta.png");

In [ ]:
1200000./4450000.

In [ ]:
TCanvas c5;
auto thetas_cumulative = thetas->GetCumulative(false);
thetas_cumulative->Scale(1./thetas->Integral(), "nosw2");
thetas_cumulative->SetTitle("Cumulative #theta;#theta");
thetas_cumulative->Draw();
c5.Draw();

In [ ]:
TCanvas c6;
auto ctaus_cumulative = ctaus->GetCumulative(false);
ctaus_cumulative->Scale(1./ctaus->Integral(), "nosw2");
ctaus_cumulative->SetTitle("Cumulative c#tau; c#tau");
ctaus_cumulative->Draw();
c6.Draw();